![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## MapReduce

### Map, reduce (i filter) in Python

Nazwa **MapReduce** pochodzi od funkcji *map* i *reduce*.  

*Map* i *reduce* (czasami nazywany *fold*) są typowe dla *programowania funkcyjnego*. 

In [1]:
# Typowa definicja funkcji w Python
def squared(x):
    return x**2

In [2]:
squared

<function __main__.squared>

In [3]:
squared(12)

144

In [4]:
# funkcja anonimowa zwana też lambda
lambda x: x**2

<function __main__.<lambda>>

In [5]:
# Referencje do lambdy można przypisać do zmiennej
squared_bis = lambda x: x**2

In [6]:
squared_bis(12)

144

In [7]:
def do_something(a, f):
    return f(a)

In [9]:
do_something(3, lambda x: x**2)

9

In [10]:
do_something(4, squared)

16

In [12]:
map?

In [13]:
# map - wykonaj funkcje na wszystkich elementach kolekcji (iterable)
# map(function, iterable)
map(squared, [1, 2, 3, 4, 5])

In [14]:
# map w Python 3 zwraca iterator; żeby otrzymać listę trzeba wykonać poniższe
list(map(squared, [1, 2, 3, 4, 5]))

[1, 4, 9, 16, 25]

In [15]:
a = [1, 2, 3]
b = list(map(squared, a))
a, b

([1, 2, 3], [1, 4, 9])

In [18]:
# W Python 3 reduce przesunięte zostało do functools
from functools import reduce 

# reduce - redukuje kolekcje do jednego elementu za pomocą funkcji (np. suma, )
# reduce(function, iterable, accumulator=None)
reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])

15

In [19]:
# filter - filtruje kolekcje używając funkcji filtrującej zwracającej wartość boolowską
# filter(function, iterable)
filter(lambda x: x % 2 == 1, [1, 2, 3, 4, 5])

In [20]:
# Podobnie, w Python 3 dostajemy iterator; należy użyć listy aby otrzymać kolekcję
list(filter(lambda x: x % 2 == 1, [1, 2, 3, 4, 5]))

[1, 3, 5]

### Zadania

Używająć `map`, `filter` i `reduce`, otrzymaj:

* Iloczyn `[1, 2, 3, 4, 5]`.
* Długość każdego słowa w liście `["Python", "Spark", "Big", "Data", "ML", "scikit-learn"]`.
* (★) Sumę wszystkich liter w słowach z powyższej listy nie zawierających litery `"i"`.

In [21]:
reduce(lambda x, y: x * y, [1, 2, 3, 4, 5])

120

In [24]:
words = ["Python", "Spark", "Big", "Data", "ML", "scikit-learn"]
m = map(lambda w: len(w), words)
list(m)

[6, 5, 3, 4, 2, 12]

In [34]:
f1 = filter(lambda w: w.lower().find('i') < 0, words)
m1 = map(lambda w: len(w), f1)
reduce(lambda x,y: x+y, m1)

17

In [35]:
# Przy okazji
x = range(1000000)

In [36]:
%%timeit
y = filter(lambda x: x % 2 == 1, x)

The slowest run took 6.80 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 249 ns per loop


In [37]:
%%timeit
y = [each for each in x if each % 2 == 1]

10 loops, best of 3: 77 ms per loop


## MapReduce in Hadoop

W Hadoop MapReduce realizowane jest z użyciem par klucz-wartość. Zobacz poniższy przykład:
![](imgs/MapReduce_example.png)

In [38]:
import sys
# Python 2
#from StringIO import StringIO
# Python 3
from io import StringIO
import contextlib

# Funkcja pomocnicza przechwytująca strumień wyjściowy
@contextlib.contextmanager
def stdoutIO(stdout=None):
    old = sys.stdout
    if stdout is None:
        stdout = StringIO()
    sys.stdout = stdout
    yield stdout
    sys.stdout = old

# Linie wejściowe do przetworzenia
lines = ['123199901', '567200806', '645200811', '989199933', '452199904', '224200822']

# Mapper wyciągający rok i liczbę
def mapper(lines):
    for line in lines:
        key = int(line[3:7])
        value = int(line[7:])
        print("{0}<>{1}".format(key, value))

# Reducer liczący sumę
def reducer(lines):
    lastKey = None
    reduce_sum = 0
    for line in lines: 
        key, value = line.split("<>")
        if lastKey is None:
            lastKey = key
        if key != lastKey:
            print("{0},{1}".format(lastKey, reduce_sum))
            reduce_sum = 0

        reduce_sum += int(value)
        lastKey = key
    print("{0},{1}".format(lastKey, reduce_sum))
    
# Przebieg MapReduce 
# Input
print("Input: {}".format(lines))
# Map
with stdoutIO() as mapper_out:
    mapper(lines)
shuffled = mapper_out.getvalue().strip().split('\n')
print("Mapper out: {}".format(shuffled))
# Shuffle
shuffled.sort()
print("Shuffeled mapper out: {}".format(shuffled))
# Reduce
with stdoutIO() as reducer_out:
    reducer(shuffled)
# Output
output = reducer_out.getvalue().strip().split('\n')
print("Output: {}".format(output))

Input: ['123199901', '567200806', '645200811', '989199933', '452199904', '224200822']
Mapper out: ['1999<>1', '2008<>6', '2008<>11', '1999<>33', '1999<>4', '2008<>22']
Shuffeled mapper out: ['1999<>1', '1999<>33', '1999<>4', '2008<>11', '2008<>22', '2008<>6']
Output: ['1999,38', '2008,39']




## Podstawy Spark

In [39]:
import pyspark

In [44]:
# Stara metoda użwająca SparkContext 
#sc = pyspark.SparkContext(appName="sparkMapReduce")
# 
# Teraz raczej używamy SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("sparkMapReduce") \
    .getOrCreate()

# Tworzymy referencje do SparkContext dla dalszej wygody
sc = spark.sparkContext

In [45]:
# RDD - Resilient Distributed Datasets, rozpraszanie danych w Spark
rdd = sc.parallelize(range(10))

In [47]:
# Obiekt a nie właściwe dane
rdd

PythonRDD[3] at RDD at PythonRDD.scala:48

In [48]:
# zwraca wszystkie elementy; należy używać z rozwagą
rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [49]:
# zwraca liczbę elementów
rdd.count()

10

In [50]:
# zwraca pierwszy
rdd.first()

0

In [63]:
# zwraca 5 pierwszych elementów
rdd.take(5)

[0, 1, 2, 3, 4]

In [62]:
# zwraca 5 elementów z najwyższą wartością
rdd.top(5)

[9, 8, 7, 6, 5]

In [61]:
# bierze próbkę losową 3 elementów (bez zwracania próbek)
rdd.takeSample(False, 3)

[8, 6, 1]

In [67]:
# operacje można łączyć w łańcuch
rdd.filter(lambda x: x % 2 == 0).collect()

[0, 2, 4, 6, 8]

In [68]:
# suma wszystkich elementów
rdd.reduce(lambda x, y: x + y)

45

In [69]:
# alternatywnie możemy wykorzystać operator
from operator import add
rdd.reduce(add)

45

In [70]:
# kolejny przykład z liczeniem słów
animals = sc.parallelize(["cat", "python", "cat", "snake", "snake"])

In [71]:
# mapa do pary klucz-wartość
animal_kv = animals.map(lambda x: (x, 1))

In [75]:
animal_kv.collect()

[('cat', 1), ('python', 1), ('cat', 1), ('snake', 1), ('snake', 1)]

In [76]:
# redukujemy parami wszystkie wartości dla danego klucza
animal_kv \
  .reduceByKey(add)  \
  .collect()

[('cat', 2), ('python', 1), ('snake', 2)]

## Text processing

Będziemy analizować ["The Tragedy of Titus Andronicus" by William Shakespeare](http://www.gutenberg.org/cache/epub/1106/pg1106.txt) z Project Gutenberg.

Zbiór nie jest Big Data ale ilustruje koncepcję przetwarzania w Spark.

In [77]:
lines = sc.textFile("data/titus_andronicus.txt")

In [78]:
# liczba linii
lines.count()

3255

In [79]:
lines.take(5)

['',
 'This Etext file is presented by Project Gutenberg, in',
 'cooperation with World Library, Inc., from their Library of the',
 'Future and Shakespeare CDROMS.  Project Gutenberg often releases',
 'Etexts that are NOT placed in the Public Domain!!']

In [81]:
# zmieniamy linie w kolekcje słów
words = lines.flatMap(lambda x: x.split())

In [82]:
words.take(5)

['This', 'Etext', 'file', 'is', 'presented']

In [83]:
# liczba słów
words.count()

23531

### Zadanie

* Co się stanie jak użyjemy `map` zamiast `flatMap`?

In [84]:
# zmieniamy linie w kolekcje słów
lines.map(lambda x: x.split()).take(5)

[[],
 ['This',
  'Etext',
  'file',
  'is',
  'presented',
  'by',
  'Project',
  'Gutenberg,',
  'in'],
 ['cooperation',
  'with',
  'World',
  'Library,',
  'Inc.,',
  'from',
  'their',
  'Library',
  'of',
  'the'],
 ['Future',
  'and',
  'Shakespeare',
  'CDROMS.',
  'Project',
  'Gutenberg',
  'often',
  'releases'],
 ['Etexts', 'that', 'are', 'NOT', 'placed', 'in', 'the', 'Public', 'Domain!!']]

In [85]:
# zmieniamy linie w kolekcje słów
lines.map(lambda x: x.split()).count()

3255

In [86]:
# Liczymy słowa tylko z pierwszą wielką literą
capitalized = words \
  .filter(lambda x: x[0].isupper()) \

In [87]:
capitalized.take(5)

['This', 'Etext', 'Project', 'Gutenberg,', 'World']

In [88]:
capitalized.distinct().take(10)

['Etext',
 'Project',
 'Gutenberg,',
 'World',
 'Library,',
 'Inc.,',
 'Future',
 'Shakespeare',
 'Public',
 'Domain!!']

In [89]:
capitalized \
  .map(lambda x: (x, 1)) \
  .reduceByKey(add) \
  .top(10, lambda x: x[1])  # możemy wybrać wartość z pary klucz-wartość jako element sortujący

[('I', 390),
 ('And', 289),
 ('TITUS.', 118),
 ('The', 90),
 ('To', 85),
 ('That', 84),
 ('MARCUS.', 65),
 ('But', 61),
 ('AARON.', 58),
 ('For', 55)]

### Zadania

* Wypisz 5 linii zaczynających się od "Titus" lub "Marcus" (usuwając spacje jeżeli potrzeba).
* Wypisz 20 najpopularniejszych słów z samymi WIELKIMI LITERAMI.
* ★ Jaka jestczęstotliwość wyrazów w dziele?

### (Python) hints

In [90]:
"  some string with whitespaces \t  ".strip()

'some string with whitespaces'

In [91]:
"Jake likes his dog.".startswith("Anne")

False

In [92]:
"Jake likes his dog.".startswith("Jake")

True

In [93]:
"Anne" or "Jake"  # Nie rób: string.startswith(a or b)

'Anne'

In [94]:
"abc,-".replace(",", "")

'abc-'

In [ ]:
"abc,-".replace(",", "").replace("-", "")

In [100]:
# Wyrażenia regularne w pythonie
import re
re.findall("[\w]+", "Titus Andronicus Roman-legion $")

['Titus', 'Andronicus', 'Roman', 'legion']

In [102]:
def starts_with_name(line):
    l = line.strip().lower()
    return l.startswith('titus') or l.startswith('marcus')

In [103]:
lines.filter(starts_with_name).take(5)

['  TITUS ANDRONICUS, a noble Roman',
 '  MARCUS ANDRONICUS, Tribune of the People, and brother to Titus',
 '  MARCUS. Princes, that strive by factions and by friends',
 '  TITUS. Hail, Rome, victorious in thy mourning weeds!',
 '    Titus, unkind, and careless of thine own,']

In [107]:
words.filter(lambda w: w.isupper()) \
    .map(lambda w: (w, 1)) \
    .reduceByKey(add) \
    .top(20, key=lambda t: t[1])

[('I', 390),
 ('TITUS.', 118),
 ('MARCUS.', 65),
 ('AARON.', 58),
 ('LUCIUS.', 51),
 ('SATURNINUS.', 50),
 ('TAMORA.', 49),
 ('A', 45),
 ('DEMETRIUS.', 39),
 ('O,', 37),
 ('OF', 36),
 ('OR', 36),
 ('FOR', 32),
 ('O', 30),
 ('CHIRON.', 30),
 ('BY', 27),
 ('AND', 27),
 ('ARE', 22),
 ('IS', 20),
 ('ELECTRONIC', 18)]